In [1]:
import os
import sys
projectDir = '/mnt/c/Users/Julien Fournier/Documents/GitHub/RakutenTeam'
sys.path.append(projectDir)

#Define path in the config file
import src.config as config
config.path_to_project = projectDir #directory of the project
config.path_to_data = os.path.join(projectDir, 'data', 'clean') #path to the data (dataframe)
config.path_to_results = os.path.join('/mnt/g/My Drive/DST/DST-Rakuten Project', 'results') #path to where the summary of the benchmark results will be saved (csv)
config.path_to_images = '/home/jul/DST/Rakuten/Data/images/image_train_resized' #path to the folder containing images
config.path_to_models = '/mnt/c/Users/Julien Fournier/Documents/DST/RakutenProject/models' #path to the folder where the models will be saved
config.path_to_tflogs = os.path.join('/mnt/g/My Drive/DST/DST-Rakuten Project', 'tf_logs') #Path to where tensorboard logs will be saved

import pandas as pd
import numpy as np

import tensorflow as tf

from Rakuten_preprocessing import Rakuten_img_path

from src.text.classifiers import TFbertClassifier

from src.utils.batch import fit_save_all
from src.utils.plot import plot_training_history

import datetime


2024-03-11 00:26:23.010971: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-11 00:26:23.061212: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 00:26:23.061241: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 00:26:23.062796: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 00:26:23.073983: I tensorflow/core/platform/cpu_feature_guar

In [2]:
data_train = pd.read_csv(os.path.join(config.path_to_data, 'df_train_index.csv'))
data_train['testset'] = False
data_test = pd.read_csv(os.path.join(config.path_to_data, 'df_test_index.csv'))
data_test['testset'] = True
data = pd.concat([data_train, data_test], axis=0)

#merging text into token column
colnames = ['designation_translated', 'description_translated'] #['designation', 'description']#
data['tokens'] = data[colnames].apply(lambda row: ' '.join(s.lower() for s in row if isinstance(s, str)), axis=1)

#path to images into img_path column
data['img_path'] = Rakuten_img_path(img_folder=config.path_to_images,
                             imageid=data['imageid'], productid=data['productid'], suffix='_resized')


In [3]:
#labels of encoded classes
class_labels = data.groupby('prdtypedesignation')['prdtypeindex'].first().reset_index()
class_labels.index = class_labels['prdtypeindex']
class_labels = class_labels.drop(columns='prdtypeindex').sort_index()

## Creating train and test sets

In [4]:
Img_train = data.loc[~data['testset'], 'img_path']
Img_test = data.loc[data['testset'], 'img_path']

Txt_train = data.loc[~data['testset'], 'tokens']
Txt_test = data.loc[data['testset'], 'tokens']

y_train = data.loc[~data['testset'],'prdtypeindex']
y_test = data.loc[data['testset'],'prdtypeindex']

#To be fed into any of our sklearn classifiers, X_train and X_test
#should be dataframes with columns tokens and img_path
X_train = pd.DataFrame({'tokens': Txt_train, 'img_path': Img_train})
X_test = pd.DataFrame({'tokens': Txt_test, 'img_path': Img_test})

#All data for cross-validated scores
X = pd.concat([X_train, X_test], axis=0)
y = pd.concat([y_train, y_test], axis=0)

#Number of classes
num_classes = len(np.unique(data['prdtypeindex']))

## Bert benchmarks

In [5]:
#Name of the summary csv file to save results to
result_file_name = 'results_benchmark_bert.csv'

#Type of classifier
class_type = 'TFbertClassifier'

#training parameters (or list of parameters for gridsearchCV)
num_class = 27
max_length = 256
n_epochs = 8
batch_size = 32
drop_rate = 0.2
lr0 = 5e-5
lr_decay_rate = 0.8

callbacks.append(('EarlyStopping', {'monitor': 'val_accuracy', 'min_delta': 0, 'mode': 'max', 'patience': 2, 'restore_best_weights': True, 'verbose': 1}))

#Adding tensorboard callback as the last one
callbacks.append(('TensorBoard', {'log_dir': np.nan, 'histogram_freq': 1, 'update_freq': 'epoch'}))

#grid search number of folds
nfolds_grid = 0

#cross-validation of f1-score
nfolds_cv = 0

#network to test
base_name_list = ['camembert-base', 'camembert-base-ccnet']

#Initializing the list of parameters to batch over
params_list = []

for base_name in base_name_list:
  #Adjusting tensorboard log directory
  log_dir = os.path.join(config.path_to_tflogs, base_name, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  callbacks[-1][1]['log_dir'] = log_dir
  #adding the set of parameters to the list
  params_list.append({'modality': 'text',
                      'class': class_type,
                      'base_name': base_name,
                      'param_grid': {'max_length': max_length, 'num_class': num_class, 'drop_rate': drop_rate, 
                                    'epochs': n_epochs, 'batch_size': batch_size, 'learning_rate': lr0, 'lr_decay_rate': lr_decay_rate
                                    'validation_data': (X_test, y_test), 'callbacks': [callbacks]},
                      'nfolds_grid': nfolds_grid, 'nfolds_cv': nfolds_cv
                    })
  
#Running the batch over params_list
results = fit_save_all(params_list, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, result_file_name = result_file_name)

Fitting:  camembert-base nan


2024-03-11 00:27:12.514818: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 00:27:12.515197: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 00:27:12.606950: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 00:27:12.607044: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 00:27:12.607093: I external/local_xla/xla/stream_executor

loading from Local


All model checkpoint layers were used when initializing TFCamembertModel.

All the layers of TFCamembertModel were initialized from the model checkpoint at /mnt/c/Users/Julien Fournier/Documents/DST/RakutenProject/models/base_models/camembert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertModel for predictions without further training.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2024-03-11 00:28:10.739458: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/8


2024-03-11 00:28:39.876585: I external/local_xla/xla/service/service.cc:168] XLA service 0x7eff6c3160e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-11 00:28:39.876704: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2024-03-11 00:28:39.876728: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA RTX A5000, Compute Capability 8.6
2024-03-11 00:28:39.891218: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-11 00:28:40.561543: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1710113320.739725 3289131 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2123/2123 [==============================] - 1573s 719ms/step - loss: 0.8086 - accuracy: 0.7839 - val_loss: 0.4817 - val_accuracy: 0.8576 - lr: 4.0000e-05
Epoch 2/8
2123/2123 [==============================] - 1507s 710ms/step - loss: 0.4156 - accuracy: 0.8756 - val_loss: 0.4293 - val_accuracy: 0.8693 - lr: 4.0000e-05
Epoch 3/8
2123/2123 [==============================] - 1505s 709ms/step - loss: 0.2988 - accuracy: 0.9089 - val_loss: 0.4300 - val_accuracy: 0.8776 - lr: 4.0000e-05
Epoch 4/8
2123/2123 [==============================] - 1518s 715ms/step - loss: 0.2229 - accuracy: 0.9323 - val_loss: 0.4558 - val_accuracy: 0.8765 - lr: 4.0000e-05
Epoch 5/8
2123/2123 [==============================] - 1529s 720ms/step - loss: 0.1656 - accuracy: 0.9498 - val_loss: 0.4835 - val_accuracy: 0.8797 - lr: 4.0000e-05
Epoch 6/8
2123/2123 [==============================] - 1525s 718ms/step - loss: 0.1299 - accuracy: 0.9603 - val_loss: 0.5121 - val_accuracy: 0.8784 - lr: 4.0000e-05
Epoch 7/8
2123/2123 

All model checkpoint layers were used when initializing TFCamembertModel.

All the layers of TFCamembertModel were initialized from the model checkpoint at /mnt/c/Users/Julien Fournier/Documents/DST/RakutenProject/models/base_models/camembert-base-ccnet.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertModel for predictions without further training.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Epoch 1/8
2123/2123 [==============================] - 1548s 711ms/step - loss: 0.7393 - accuracy: 0.7824 - val_loss: 0.4902 - val_accuracy: 0.8480 - lr: 4.0000e-05
Epoch 2/8
2123/2123 [==============================] - 1526s 719ms/step - loss: 0.4154 - accuracy: 0.8713 - val_loss: 0.4383 - val_accuracy: 0.8685 - lr: 4.0000e-05
Epoch 3/8
2123/2123 [==============================] - 1538s 724ms/step - loss: 0.3000 - accuracy: 0.9057 - val_loss: 0.4575 - val_accuracy: 0.8666 - lr: 4.0000e-05
Epoch 4/8
2123/2123 [==============================] - 1516s 714ms/step - loss: 0.2230 - accuracy: 0.9291 - val_loss: 0.4644 - val_accuracy: 0.8760 - lr: 4.0000e-05
Epoch 5/8
2123/2123 [==============================] - 1517s 714ms/step - loss: 0.1669 - accuracy: 0.9474 - val_loss: 0.5175 - val_accuracy: 0.8709 - lr: 4.0000e-05
Epoch 6/8
2123/2123 [==============================] - 1537s 724ms/step - loss: 0.1324 - accuracy: 0.9581 - val_loss: 0.5367 - val_accuracy: 0.8747 - lr: 4.0000e-05
Epoch 6: e

## Visualize tensorflow logs in tensorboard

In [15]:
log_path = os.path.join(config.path_to_tflogs, 'camembert-base-ccnet')

# Ensure the log_path is quoted to handle spaces
quoted_log_path = f'"{log_path}"'

%reload_ext tensorboard
%tensorboard --logdir {quoted_log_path}

Reusing TensorBoard on port 6008 (pid 3180493), started 0:00:03 ago. (Use '!kill 3180493' to kill it.)

## Example usage

In [ ]:
#defining callbacks
callbacks = []
callbacks.append(('EarlyStopping', {'monitor': 'val_accuracy', 'min_delta': 0, 'mode': 'max', 'patience': 2, 'restore_best_weights': True, 'verbose': 1}))

clf = TFbertClassifier(base_name='camembert-base', from_trained=None, max_length=256, num_class=27, epochs=8, batch_size=32, drop_rate=0.2, learning_rate=5e-5, lr_decay_rate=0.6, validation_data=(X_test, y_test), callbacks=callbacks)

clf.fit(X_train, y_train)
# clf.classification_score(X_test, y_test)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2024-03-10 13:07:20.850430: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.850632: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.948389: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.948489: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.948545: I external/local_xla/xla/stream_executor

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu